## A. Configuración General.

In [2]:
#1. Librerías.
%run "../librerias.ipynb"

Matplotlib is building the font cache; this may take a moment.


/home/ianlink95/.venv/lib/python3.12/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
/home/ianlink95/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#3. Constantes.
%run "../constantes.ipynb"

dataset_con_fe = dataset_file_fe_all_1

cantidad_meses_train = "all"
ventana = 1

mes_train = mes_train_all_menos_1
mes_test = mes_test

In [4]:
#4. Funciones
%run "../funciones.ipynb"

In [5]:
#5. Lectura de datos.
data = pd.read_parquet(dataset_con_fe)

In [6]:
#6. Pequeño pre-procesamiento sobre los datos.
#i. Cambio tipos de datos (Me lo toma como tipo de dato "object"...)
data['ctrx_quarter_normalizado'] = data['ctrx_quarter_normalizado'].astype(float)
#ii. Elimino columnas de último momento por Data Concept.
columnas_de_interes_prestamos = data.filter(like='prestamos_personales').columns
data.drop(columnas_de_interes_prestamos,axis=1,inplace=True)
#iii. Pesos y reclusterización.
data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

data['clase_binaria2'] = 0
data['clase_binaria2'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1)

In [7]:
#iv. Divido entre Train y Test.
train_data = data[data['foto_mes'].isin(mes_train)]
test_data = data[data['foto_mes'] == mes_test]

In [8]:
#v. Undersampleo.
#a. Filtramos las clases.
continua_train = train_data[(train_data['clase_binaria2'] == 0)]
baja_train = train_data[(train_data['clase_binaria2'] == 1)]

#b. Lista para almacenar los datos submuestreados.
continua_undersampleados = []

#c. Iteramos sobre cada mes para submuestrear.
for mes in continua_train['foto_mes'].unique():
    #1. Filtramos los datos de la clase mayoritaria para el mes específico.
    continua_mes_seleccionado = continua_train[continua_train['foto_mes'] == mes]
    
    #2. Calculamos el tamaño objetivo para el undersampling (30%).
    n_samples = int(len(continua_mes_seleccionado) * 0.3)
    
    #3. Submuestreamos las instancias de la clase mayoritaria para este mes
    continua_mes_seleccionado_undersampleados = resample(continua_mes_seleccionado, 
                                        replace=False, 
                                        n_samples=n_samples, 
                                        random_state=semillas[0])
    
    #4. Agregamos los datos submuestreados a la lista
    continua_undersampleados.append(continua_mes_seleccionado_undersampleados)

#d. Concatenamos todos los meses submuestreados en un solo DataFrame
continua_undersampleados = pd.concat(continua_undersampleados)

#e. Concatenamos la clase mayoritaria submuestreada con la clase minoritaria completa
train_undersampleado = pd.concat([continua_undersampleados, baja_train])

#v. Separar en X e y después del undersampling.
#a. Datos para optimizar Optuna.
X_train_undersampleado = train_undersampleado.drop(['clase_ternaria', 'clase_peso', 'clase_binaria2'], axis=1)
y_train_binaria2_undersampleado = train_undersampleado['clase_binaria2']
w_train_undersampleado = train_undersampleado['clase_peso']

#b. Datos para entrenar todo el modelo final para Kaggle.
X_train = train_data.drop(['clase_ternaria', 'clase_peso','clase_binaria2'], axis=1)
y_train_binaria2 = train_data['clase_binaria2']
w_train = train_data['clase_peso']

#c. Datos de Test (a predecir).
X_test = test_data.drop(['clase_ternaria', 'clase_peso','clase_binaria2'], axis=1)

## B. Train con cantidad_meses_train meses con df -ventana con ratios incluidos.

In [9]:
#1. Funcion de optimización de hiperparámetros.
def objective(trial): 
    # Rango de parámetros a buscar sus valores óptimos.
    num_leaves = trial.suggest_int('num_leaves', 10, 200)
    learning_rate = trial.suggest_float('learning_rate', 0.005, 0.3) # mas bajo, más iteraciones necesita.
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 15, 900)
    feature_fraction = trial.suggest_float('feature_fraction', 0.1, 1.0)
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.1, 1.0)


    # Parámetros que le voy a pasar al modelo.
    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'seed': semillas[0],
        'verbose': -1
    }
    
    # Creo el dataset para Light GBM.
    train_data_ob = lgb.Dataset(X_train_undersampleado,
                              label=y_train_binaria2_undersampleado, # eligir la clase
                              weight=w_train_undersampleado)
    
    # Entreno.
    cv_results = lgb.cv(
        params,
        train_data_ob,
        num_boost_round=1000, # modificar, subit y subir... y descomentar la línea inferior
        callbacks=[lgb.early_stopping(int(50 + 5 / learning_rate))],
        feval=lgb_gan_eval,
        stratified=True,
        nfold=5,
        seed=semillas[0]
    )
    
    # Calculo la ganancia máxima y la mejor iteración donde se obtuvo dicha ganancia.
    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    # Guardamos cual es la mejor iteración del modelo
    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5

In [ ]:
#2. Voy a realizar un estudio de Optuna para encontrar los mejores parámetros.
#i. Creo la base de datos donde guardar los resultados.
storage_name = "sqlite:///" + db_path + "optimization_lgbm.db"

study_name = f"exp_lgbm_{cantidad_meses_train}_{ventana}_undersampling" # Primer dígito cuantos meses para atrás desde 06/21, segundo dígito número data drifting.

#ii. Creo el estudio.
study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

#iii. Corro el estudio.
study.optimize(objective, n_trials=100)

[I 2024-11-09 02:30:39,257] A new study created in RDB with name: exp_lgbm_all_1_undersampling


Training until validation scores don't improve for 88 rounds
Early stopping, best iteration is:
[223]	cv_agg's valid gan_eval: 6.48021e+08 + 9.7715e+06


[I 2024-11-09 02:54:31,809] Trial 0 finished with value: 3240104000.0 and parameters: {'num_leaves': 121, 'learning_rate': 0.12918106155967501, 'min_data_in_leaf': 639, 'feature_fraction': 0.6391425543002685, 'bagging_fraction': 0.3842259584712716}. Best is trial 0 with value: 3240104000.0.


Training until validation scores don't improve for 70 rounds
Early stopping, best iteration is:
[37]	cv_agg's valid gan_eval: 6.13962e+08 + 1.22272e+07


[I 2024-11-09 03:01:26,333] Trial 1 finished with value: 3069808000.0 and parameters: {'num_leaves': 71, 'learning_rate': 0.24950053842993855, 'min_data_in_leaf': 27, 'feature_fraction': 0.18653538911882572, 'bagging_fraction': 0.5503657943132991}. Best is trial 0 with value: 3240104000.0.


...

Training until validation scores don't improve for 162 rounds
Did not meet early stopping. Best iteration is:
[977]	cv_agg's valid gan_eval: 6.5813e+08 + 7.72732e+06


[I 2024-11-09 22:06:48,660] Trial 27 finished with value: 3290651000.0 and parameters: {'num_leaves': 183, 'learning_rate': 0.044272968270519034, 'min_data_in_leaf': 151, 'feature_fraction': 0.2427752497270366, 'bagging_fraction': 0.44925140426522275}. Best is trial 14 with value: 3303951000.0.


Training until validation scores don't improve for 77 rounds
Early stopping, best iteration is:
[238]	cv_agg's valid gan_eval: 6.38497e+08 + 8.26628e+06


[I 2024-11-09 22:19:59,883] Trial 28 finished with value: 3192483000.0 and parameters: {'num_leaves': 129, 'learning_rate': 0.18285025171265484, 'min_data_in_leaf': 371, 'feature_fraction': 0.9682282307464434, 'bagging_fraction': 0.32350728661133754}. Best is trial 14 with value: 3303951000.0.


Training until validation scores don't improve for 83 rounds


In [ ]:
#4. Visualizo los resultados del estudio, para modificar los rangos de análisis.

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
plot_param_importances(study)

In [ ]:
plot_slice(study)

In [ ]:
plot_contour(study)

In [ ]:
plot_contour(study, params=['num_leaves','min_data_in_leaf'] )

In [ ]:
study.best_trial.params

In [ ]:
#5. Tomamos el mejor modelo y con eso entrenamos todos los datos.
best_iter = study.best_trial.user_attrs["best_iter"]
print(f"Mejor cantidad de árboles para el mejor model {best_iter}")
params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_bin': 31,
    'num_leaves': study.best_trial.params['num_leaves'],
    'learning_rate': study.best_trial.params['learning_rate'],
    'min_data_in_leaf': study.best_trial.params['min_data_in_leaf'],
    'feature_fraction': study.best_trial.params['feature_fraction'],
    'bagging_fraction': study.best_trial.params['bagging_fraction'],
    'seed': semillas[0],
    'verbose': 0
}

train_data_modelo = lgb.Dataset(X_train,
                          label=y_train_binaria2,
                          weight=w_train)

model_lgb = lgb.train(params,
                  train_data_modelo,
                  num_boost_round=best_iter)

In [ ]:
#6. Observamos las variables más importantes para el modelo.
#i. Gráfico.
lgb.plot_importance(model_lgb, figsize=(10, 20))
plt.show()

In [ ]:
#ii. Dataframe.
#a. Extract feature importance and feature names.
importance = model_lgb.feature_importance()
features = model_lgb.feature_name()

#b. Create a dataframe for better visualization.
importance_df = pd.DataFrame({'Feature': features, 'Importance': importance})

#c. Sort by importance in descending order.
importance_df = importance_df.sort_values(by='Importance', ascending=False).reset_index(drop=True)

#d. show.
importance_df.head(100)

In [ ]:
#7. Guardamos el modelo.
# Primer dígito cuantos meses para atrás desde 06/21, segundo dígito número data drifting, tercer dígito número de entrenamiento.
model_lgb.save_model(modelos_path + 'lgbm_{}_{}_undersampling.txt'.format(cantidad_meses_train,ventana))

In [ ]:
####################################################################################################
####################################################################################################
####################################################################################################
####################################################################################################
############## Primera subida orientativa a Kaggle previo medir semillas, etc. #####################
####################################################################################################
####################################################################################################
####################################################################################################
####################################################################################################

In [ ]:
#8. Volvemos a leer el modelo.
model_lgb = lgb.Booster(model_file= modelos_path + 'lgbm_{}_{}_undersampling.txt'.format(cantidad_meses_train,ventana))

In [ ]:
#a. Importo librería.
from kaggle.api.kaggle_api_extended import KaggleApi
#b. Configura el API de Kaggle
api = KaggleApi()
api.authenticate()

In [ ]:
#L. Predecimos Agosto.
#i. Predecimos propiamente dicho.
predicciones = model_lgb.predict(X_test)
#ii. Le pegamos la probabilidad de ser "BAJA" a cada cliente.
X_test['Probabilidad'] = predicciones
#iii. Ordenamos a los clientes por probabilidad de ser "BAJA" de forma descendente.
tb_entrega = X_test.sort_values(by='Probabilidad', ascending=False)
#iv. Genero una lista de distintos cortes candidatos, para enviar a Kaggle.
cortes = range(9000,14000,200)
#v. Generamos las distintas predicciones de clases a partir de los distintos cortes posibles.
num_subida_kaggle = 1
i = 0
for envios in cortes:
    #1. Le ponemos clase 1 ("BAJA") a los primeros "envios" con mayor probabilidad.
    tb_entrega['Predicted'] = 0
    tb_entrega.iloc[:envios, tb_entrega.columns.get_loc('Predicted')] = 1
    resultados = tb_entrega[["numero_de_cliente", 'Predicted']].reset_index(drop=True)
    
    print("Cantidad de clientes {}".format(envios))
    #2. Guardamos el archivo para Kaggle.
    nombre_archivo = "K_OH_{}_{}_00{}.csv".format(cantidad_meses_train,ventana,num_subida_kaggle) #-X meses, con df de -X meses, número de intento.
    ruta_archivo= "{}/{}".format(exp_path,nombre_archivo)
    resultados.to_csv(ruta_archivo, index=False)
    
    num_subida_kaggle += 1
    i += 1
    
    #3. Envío a Kaggle.
    #a. Defino los parámetros claves.
    mensaje = f'Archivo {nombre_archivo}.LGBM meses train {cantidad_meses_train} con undersampling, DF {ventana}, punto_corte: {envios}.'
    competencia = 'dm-ey-f-2024-segunda'
    #c. Subo la Submission.
    while i<=15:
        print(i)
        api.competition_submit(file_name=ruta_archivo, message=mensaje, competition=competencia)
        print("Submission successful!")
        break
    else:
        print("Esperamos 30 segundos...")
        time.sleep(30)
        api.competition_submit(file_name=ruta_archivo, message=mensaje, competition=competencia)
        print("Submission successful!")
        i= 0

In [ ]:
print("Terminó")